In [10]:
### This python Notebook is for Vicki to try out and submit json queries to ARS through backup ARAX 

## You will need to install jyupter notebook and install all the packages below

In [1]:
# packages
import requests
import json
from collections import defaultdict
import os
import pandas as pd

In [2]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

In [3]:
def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    pk = 'https://arax.ncats.io/?source=ARS&id=' + mid
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    dictionary = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                #print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
            
        dictionary['pk_id'] =  pk  
            
        if ((child['status'] == 'Done') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'No Results'
            #test =  [child['actor']['agent'], 'No Results']
        elif ((child['status'] == 'ARS Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'ARS Error'
        elif ((child['status'] == 'Error') & (nresults == 0)):
            dictionary[child['actor']['agent']] = 'Error'
            #test =  [child['actor']['agent'], 'ARS Error']
        elif ((child['status'] == 'Done') & (nresults != 0)):
            #test =  [child['actor']['agent'], 'Results']
            dictionary[child['actor']['agent']] = 'Results' 
        
        
        print(child['actor']['agent'], child['status'], nresults)
        #test =  [child['actor']['agent'], child['status'], nresults]
        #test2.append(test)
    return dictionary

def printjson(j):
    print(json.dumps(j,indent=4))

<br>

## Test for individual run

<br>

In [5]:
with open('/Users/priyash/Documents/GitHub/minihackathons/2021-12_demo/workflowB/B.2_DILI-fourth-one-hop-from-disease-or-phenotypic-feature_trapi.json') as inf:
    query1 = json.load(inf)

In [6]:
printjson(query1)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "CHEBI:41879"
                    ],
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": [
                        "has_real_world_evidence_of_association_with"
                    ]
                }
            }
        }
    }
}


In [7]:
kcresult = submit_to_ars(query1)

https://arax.ncats.io/?source=ARS&id=bc950c8d-4ce2-4ded-9c8b-b8c72c9163ad


In [9]:
result_status = retrieve_ars_results(kcresult)

Done
Error
ara-aragorn Error 0
Error
ara-arax Error 0
Done
ara-bte Done 0
Unknown
ara-unsecret Unknown 0
Error
kp-genetics Error 0
Error
kp-molecular Error 0
Error
ara-explanatory Error 0
Error
ara-improving Error 0
Done
kp-cam Done 0
Done
kp-textmining Done 0
Error
ara-aragorn-exp Error 0
Error
kp-openpredict Error 0
Error
kp-icees Error 0
Error
ara-robokop Error 0
Done
kp-chp Done 0
Error
kp-cohd Error 0
Error
kp-icees-dili Error 0
